# Yet Another OpenPose 

---
## Environment

In [1]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which Environment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

desktop-sj3un3d\bfgydx
Python running from: E:\Anaconda3\envs\YAOI
Current working dir E:\DeepLearning\Yet-Another-Openpose-Implementation-master\training


'gcloud' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

In [3]:
#start TPU
!gcloud compute tpus start node-1 --zone us-central1-c
#stop TPU
#!gcloud compute tpus stop node-1 --zone us-central1-c

'gcloud' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [4]:

## Config

SyntaxError: invalid syntax (Temp/ipykernel_5512/1412333348.py, line 1)

In [2]:
import sys
sys.path.append("E:/DeepLearning/Yet-Another-Openpose-Implementation-master/configs/")

import default_config as cfg

import local_storage_config as storage_cfg
#import remote_storage_config as storage_cfg
cfg.__dict__.update(storage_cfg.__dict__)

import tpu_training.TPU_config as TPU_config
cfg.__dict__.update(TPU_config.__dict__) #comment out to disable TPU training

cfg.RUN_NAME="model11" #for reference 

In [3]:
from utils import now
nowt=now()

In [4]:
import tensorflow as tf
print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.1.0


In [ ]:
if cfg.TPU_MODE:
    import tpu_training.init_TPU as init_TPU
    
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {cfg.TENSORBOARD_PATH}/test
    !gsutil rm {cfg.TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {cfg.CHECKPOINTS_PATH}/test
    !gsutil rm {cfg.CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{cfg.GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {cfg.TPU_IP}
    strategy,resolver=init_TPU.init_tpu(cfg.TPU_IP) #This must be run before any imports!!!!

Testing results bucket connectivity


'touch' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'gsutil' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'gsutil' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'gsutil' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Testing dataset bucket connectivity


'gsutil' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'gsutil' 不是内部或外部命令，也不是可运行的程序

Testing TPU connectivity



或批处理文件。


Trying to connect to a TPU node

!!!MAKE SURE THE TPU ADDRESS IS CORRECT!!
1.ip must be correct
2.tpu must be turned on
3.version must be 'nightly-2.x'
4.tpu must be reachable (check with gce networking/connectivity test)
if not this will hang!

Trying to connect to: grpc://10.0.3.2:8470


In [ ]:
# import datetime
import numpy as np

import dataset_functions
import models.six_stage_linear_model as model
import callbacks
import dataset_builder
import load_weights
import loss_metrics

In [ ]:
# import importlib as il
# il.reload() #useful to reload any changes

---
## Dataset

In [ ]:
# Training settings
TRAINING_EPOCHS = 30
REAL_EPOCH_STEPS = int(cfg.DATASET_SIZE / cfg.BATCH_SIZE)
SHORT_EPOCH_STEPS=50 #actual epocsh used in training, smaller than real epoch, but allows to track progress better, [in batches]
SHORT_TRAINING_EPOCHS=int(TRAINING_EPOCHS*(REAL_EPOCH_STEPS/SHORT_EPOCH_STEPS))
SHORT_VALIDATION_STEPS=5 #per short epoch

EPOCH_RATIO=int(REAL_EPOCH_STEPS / SHORT_EPOCH_STEPS)
# adam_learning_rate=0.001  #for reference
BASE_LEARNING_RATE = 0.001
LEARNING_RATE_SCHEDUELE = np.zeros(100000)  #used with short epochs
LEARNING_RATE_SCHEDUELE[:3 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[3 * EPOCH_RATIO:20 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[20 * EPOCH_RATIO:40 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[40 * EPOCH_RATIO:100 * EPOCH_RATIO] = 0.5
LEARNING_RATE_SCHEDUELE[100 * EPOCH_RATIO:] = 0.3
LEARNING_RATE_SCHEDUELE *= BASE_LEARNING_RATE

In [ ]:
model_ds=model.ModelDatasetComponent(cfg)

tfrecord_files_train=dataset_builder.get_tfrecord_filenames(cfg.TRAIN_TFRECORDS,cfg)
tfrecord_files_valid=dataset_builder.get_tfrecord_filenames(cfg.VALID_TFRECORDS,cfg)
print("Found the following training TFrecords:\n","\n".join(tfrecord_files_train))
print("Found the following validation TFrecords:\n","\n".join(tfrecord_files_valid))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train,model_ds.place_training_labels,cfg)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_valid,model_ds.place_training_labels,cfg)
print("Validation dataset shape:",dsv)


In [ ]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [ ]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
## Model
### Load weights

In [ ]:
if cfg.ASK_FOR_CHECKPOINTS:
    checkpoint,starting_epoch=load_weights.checkpoints_prompt(cfg)
else:
    checkpoint=None
    starting_epoch=0

### Build Model

In [ ]:
model_maker=model.ModelMaker(cfg) #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
#     if cfg.INCLUDE_MASK:
#         losses=[loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanSquaredError()
#                 ,loss_metrics.MaskedMeanSquaredError()]                      
#     else:
#         raise NotImplementedError       
    
    #this must match the model output order
    metrics=[
              [loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]    
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(BASE_LEARNING_RATE)                   
                    ,loss=loss_metrics.MaskedMeanSquaredError()
                    ,loss_weights=[5,5,5,5,1,1]
                    ,metrics=metrics                           
                   )
    return train_model,test_model

if cfg.TPU_MODE:
    print("TPU")
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    print("gpu")
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

In [ ]:
all_callbacks=[
    callbacks.make_LRscheduler_callback(LEARNING_RATE_SCHEDUELE)
    ,callbacks.PrintLR()
    ,tf.keras.callbacks.TerminateOnNaN()
]
if cfg.SAVE_CHECKPOINTS:
    checkpoint_callback,checkpoint_path=callbacks.make_checkpoint_callback(cfg,nowt,REAL_EPOCH_STEPS*cfg.BATCH_SIZE)
    all_callbacks.append(checkpoint_callback)
if cfg.SAVE_TENSORBOARD:
    tensorboard_callback,tensorboard_path=callbacks.make_tensorboard_callback(cfg,nowt,EPOCH_RATIO)
    all_callbacks+=[tensorboard_callback]

In [ ]:
#run to clean all tensorboard&checkpoints results
#!gsutil -m rm -r {cfg.TENSORBOARD_PATH}/*
#!gsutil -m rm -r {cfg.CHECKPOINTS_PATH}/*


---
## Training
Actually training

In [ ]:
train_history=train_model.fit(
    dst
    ,epochs=SHORT_TRAINING_EPOCHS
    ,steps_per_epoch=SHORT_EPOCH_STEPS
    ,validation_steps=SHORT_VALIDATION_STEPS
    ,validation_data=dsv
    ,callbacks=all_callbacks
    ,initial_epoch=starting_epoch
)

### Callbacks

In [ ]:
#run to clear this run results
#!gsutil -m rm -r {tensorboard_path}/*
#!gsutil -m rm -r {checkpoint_path}/*

---
### Save results

In [ ]:
tmp_path='gs://dl_training_results/tmp/tensorflow/temp_weights'
train_model.save_weights(tmp_path)

local_model_maker=model.ModelMaker(cfg) #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model

cpu_train_model,cpu_test_model=local_model_maker.create_models()

cpu_train_model.load_weights(tmp_path)
cpu_test_model.load_weights(tmp_path)

cpu_test_model.save(cfg.MODELS_PATH+"/test_"+cfg.RUN_NAME+nowt,include_optimizer=False) 
cpu_train_model.save(cfg.MODELS_PATH+"/train_"+cfg.RUN_NAME+nowt,include_optimizer=False) 

---
### Shutdown

In [ ]:
#shut down TPU
!gcloud compute tpus stop node-1 --zone us-central1-c

In [ ]:
#shut down this host
!gcloud compute instances stop instance-1 --zone us-central1-c